<a href="https://colab.research.google.com/github/anadebattista/TMnews/blob/master/TMnews_1_Obtencion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Curso Minería de Textos 2019**
**Trabajo Final:** 

**Profesor:** Dr. Marcelo Errecalde

**Alumnos:** 
*   M.Cs. Anabella De Battista
*   Ing. Esteban Schab

---
#**Fase de Obtención y Preparación de los Textos**

---

**Determinación de fuentes de información**

Para este trabajo se consideró trabajar con tres periódicos digitales nacionales: Clarín, Infobae y La Nación. Se seleccionaron 6 secciones de los diarios: Deportes, Economía, Espectáculos, Política, Sociedad, Tecnología.

---

**Carga de librerías**

---

In [0]:
import pandas as pd
import os
import chardet
import numpy as np
import datetime

In [0]:
! git clone https://github.com/GIBD/TMnews.git

Cloning into 'TMnews'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 537 (delta 0), reused 7 (delta 0), pack-reused 527
Receiving objects: 100% (537/537), 56.75 MiB | 20.65 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (518/518), done.


---

**Generación de un único almacén de datos**

A partir de los múltiples archivos generados al scrappear los sitios web de los diarios, se generó un único repositorio en formato CSV (newsoctubre2019.csv)

---

In [0]:
os.chdir("/content/TMnews/data/raw")
# Metadatos para acceder a los archivos JSON
diarios = ['clarin','infobae','lanacion']
secciones = ['deportes','economia','espectaculos','politica','sociedad','tecnologia']
dias = ['01','02','04','05','06','07','08','09','10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
# Inicializa dataframe noticias
noticias = pd.DataFrame()
# Recorre los archivos JSON cargando las noticias en el dataframe noticias
for diario in diarios:
    for seccion in secciones:
        for dia in dias:
            nombreJSON = '201910' + dia + '_' + diario + '_' + seccion + '.json'
            # Genera la fecha del JSON actual
            fechaJSON = datetime.datetime(int(nombreJSON[0:4]),int(nombreJSON[4:6]), int(nombreJSON[6:8]))
            # Obtiene metadatos del archivo para acceder al encoding
            with open(nombreJSON, 'rb') as f: result = chardet.detect(f.read())
            # Obtiene el dataframe a partir del JSON teniendo en cuenta el encoding
            if (result['encoding'] != 'Windows-1252'):
              df = pd.read_json(nombreJSON, lines=True) 
            else:
              df = pd.read_json(nombreJSON, lines=True,  encoding=result['encoding']) #encoding='Windows-1252'
            # Actualiza la fecha en el dataframe
            df['date'] = fechaJSON
            #concatena con los anteriores
            noticias = pd.concat([noticias, df], ignore_index=True, sort=False)
# Exporta el dataframe de noticias en formato CSV
export_csv = noticias.to_csv (r'/content/TMnews/data/newsoctubre2019.csv', index = None, header=True)
#noticias.shape
noticias

,news_paper,category,link,date,title,abstract,author,key_words,body
0,clarin,deportes,https://www.clarin.com//deportes/escandalo-aud...,2019-10-01,"“¿Por qué no te ponés a dieta?”, el escándalo ...",Se difundió un mensaje del referí Germán Bermú...,[],Federico BeligoyArbitrosPrueba,Germán Bermúdez es referí en las categorías de...
1,clarin,deportes,https://www.clarin.com//deportes/espana-hablan...,2019-10-01,En España hablan de una fractura en el Barcelo...,El diario El País dice que la contratación de ...,[],FC BarcelonaLionel MessiAntoine GriezmannNeymar,Barcelona no mira a todos desde arriba. Ya no....
2,clarin,deportes,https://www.clarin.com//deportes/superclasico-...,2019-10-01,Superclásico: qué dicen los astros para el Riv...,"En la previa del partido de ida, dos reconocid...",[],River PlateBoca JuniorsSuperclásicoCopa Libert...,"Ya sean creyentes o escépticos, la ansiedad po..."
3,clarin,deportes,https://www.clarin.com//deportes/river-boca-co...,2019-10-01,"River-Boca, por la Copa Libertadores: cómo est...",Las fuertes tormentas que cayeron sobre la ciu...,[],Copa LibertadoresSuperclásicoRiver PlateBoca J...,El duelo de ida por las semifinales de la Copa...
4,clarin,deportes,https://www.clarin.com//deportes/mundial-atlet...,2019-10-01,"El Mundial de atletismo, envuelto en el drama ...",La IAAF está en la mira por haber permitido qu...,Luciana Aranguiz,AtletismoDoha,La imagen se repitió una y otra vez en las pri...
...,...,...,...,...,...,...,...,...,...
37640,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/como-z-...,2019-10-29,Como una Z: así es el prototipo de TCL de un s...,NaN,,,TCL presentó por estos días un prototipo de un...
37641,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/lanzan-...,2019-10-29,Lanzan una app gratis para ayudar a los partid...,NaN,Sebastián Davidovsky,,Una aplicación colaborativa que les facilite a...
37642,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/youtube...,2019-10-29,YouTube presenta su primera película interacti...,NaN,,,YouTube publicó el adelanto de \r\nA Heist Wit...
37643,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/apple-p...,2019-10-29,Apple planea lanzar una función para crear Mem...,NaN,,,Los Memojis son las ilustraciones personalizad...


---

**Resumen de cantidad de noticias según diario y sección**

---

In [0]:
# Agrupa las noticias por diario y categoría
a = noticias.groupby(['news_paper', 'category'])
# Genera un array numpay solo con la cantidad de noticias
b = np.array(a.aggregate(np.count_nonzero)['link'])
# Agrega al array la sumatoria por secciones
b = np.append(b,[b[0:6] + b[6:12] + b[12:18]])
# Genera un dataframe con la cantidad de noticias por seccion y diario y los totales
resumen = pd.DataFrame({"Clarin":np.append(b[0:6],[b[0:6].sum()]), 
                        "Infobae":np.append(b[6:12],[b[6:12].sum()]),  
                        "La Nacion":np.append(b[12:18],[b[12:18].sum()]),
                        "TOTAL":np.append(b[18:24],[b[18:24].sum()])},
             index=['Deportes', 'Economía','Espectaculos','Politica','Sociedad','Tecnologia','TOTAL'])
# Muestra el dataframe
resumen.T

,Deportes,Economía,Espectaculos,Politica,Sociedad,Tecnologia,TOTAL
Clarin,1262,1260,1224,1304,1246,1260,7556
Infobae,3737,1567,2787,2175,2060,2577,14903
La Nacion,1685,1419,2776,3285,2984,3037,15186
TOTAL,6684,4246,6787,6764,6290,6874,37645


---

Se exporta la tabla resumen como archivo CSV

---

In [0]:
news_resumen = resumen.to_csv (r'/content/TMnews/data/news_resumen.csv', index = True, header=True)